In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import jax
import jax.numpy as jnp
from flax import struct
import numpy as np
from functools import partial
from typing import Optional

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [3]:
class FeedForward(nn.Module):
    def __init__(
        self,
        dim: int,
        hidden_dim: int,
        multiple_of: int,
        ffn_dim_multiplier: Optional[float],
    ):
        super().__init__()
        # hidden dim gymnastics that Meta simplified only later
        hidden_dim = int(2 * hidden_dim / 3)
        if ffn_dim_multiplier is not None:
            hidden_dim = int(ffn_dim_multiplier * hidden_dim)
        hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)
        self.w1 = nn.Linear(dim, hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, dim, bias=False)
        self.w3 = nn.Linear(dim, hidden_dim, bias=False)

    def forward(self, x):
        gate = self.w1(x)
        up = self.w3(x)
        return self.w2(F.silu(gate) * up)


In [4]:

@struct.dataclass
class FeedForwardParams:
    w_gate: jax.Array  # Corresponds to gate_proj
    w_up: jax.Array  # Corresponds to up_proj
    w_down: jax.Array  # Corresponds to down_proj

@partial(jax.jit, static_argnames=["activation_fn"],donate_argnums=[0])
def feed_forward_jax(
    x: jax.Array,
    params: FeedForwardParams,
    activation_fn: str,  # Added activation function name
) -> jax.Array:
    """
    Compute FeedForward network (MLP) using a configurable activation function (like SwiGLU).

    Args:
        x: Input tensor of shape [batch_size, seqlen, dim].
        params: Dataclass containing weight matrices (w_gate, w_up, w_down).
        activation_fn: Name of the activation function ('silu', 'relu', 'gelu').

    Returns:
        Output tensor after MLP computation.
    """

    # Project input: x -> gate, up
    # x: [bs, seqlen, dim], w_gate: [dim, hidden_dim], w_up: [dim, hidden_dim]
    gate = jnp.einsum("bsd,dh->bsh", x, params.w_gate)
    up = jnp.einsum("bsd,dh->bsh", x, params.w_up)

    # Apply the specified activation function (SwiGLU style)
    if activation_fn == "silu":
        activated_gate = jax.nn.silu(gate)
    elif activation_fn == "relu":
        activated_gate = jax.nn.relu(gate)
    elif activation_fn == "gelu":
        # Use approximate=False for exact GELU, True for faster approximation
        activated_gate = jax.nn.gelu(gate, approximate=False)
    else:
        raise ValueError(f"Unsupported activation function: {activation_fn}")
        # replace error handling with chex

    fused_activation = activated_gate * up

    # Project down
    # fused_swiglu: [bs, seqlen, hidden_dim], w_down: [hidden_dim, dim]
    output = jnp.einsum("bsh,hd->bsd", fused_activation, params.w_down)

    return output

In [5]:
bsz = 2
seqlen = 64
dim = 192
multiple_of = 32
dtype = np.float32

# Mimic hidden_dim calculation from PyTorch implementation
hidden_dim = int(2 * (4 * dim) / 3)
hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)

# 2. Create shared weights and inputs
np.random.seed(0)
x_np = np.random.randn(bsz, seqlen, dim).astype(dtype)

# PyTorch names weights w1, w2, w3. JAX uses w_gate, w_up, w_down.
# Mapping: torch.w1 -> jax.w_gate, torch.w3 -> jax.w_up, torch.w2 -> jax.w_down
w1_np = np.random.randn(dim, hidden_dim).astype(dtype)  # gate_proj
w3_np = np.random.randn(dim, hidden_dim).astype(dtype)  # up_proj
w2_np = np.random.randn(hidden_dim, dim).astype(dtype)  # down_proj

In [6]:

# 3. JAX setup
x_jax = jnp.array(x_np)
jax_params = FeedForwardParams(
    w_gate=jnp.array(w1_np), w_up=jnp.array(w3_np), w_down=jnp.array(w2_np)
)
output_jax = feed_forward_jax(x_jax, jax_params,"silu")


In [8]:
#os.environ["PJRT_DEVICE"]="TPU"
# 4. PyTorch setup
x_torch = torch.tensor(x_np, device=device)
torch_ff = FeedForward(
    dim=dim, hidden_dim=hidden_dim, multiple_of=multiple_of, ffn_dim_multiplier=None
)
torch_ff.w1.weight = torch.nn.Parameter(torch.tensor(w1_np.T, device=device))
torch_ff.w3.weight = torch.nn.Parameter(torch.tensor(w3_np.T, device=device))
torch_ff.w2.weight = torch.nn.Parameter(torch.tensor(w2_np.T, device=device))
output_torch = torch_ff(x_torch)

gate[0,:5,0]
 tensor([  2.8536,  -9.0668,  28.0873, -16.5318,  -2.2464],
       grad_fn=<SelectBackward0>)
up[0,:5,0]
 tensor([ 16.0499, -10.1857,  -5.5634, -21.5287,  -1.0652],
       grad_fn=<SelectBackward0>)
w1.weight[0,:5]
 tensor([-0.2965, -0.1555,  2.1040, -1.6559, -0.6499], grad_fn=<SliceBackward0>)


In [ ]:
print(output_jax[0,:5,0])
print(output_torch[0,:5,0])

[ 3824.567   3250.896  -1562.5957  4669.1475 -2415.9502]
tensor([ 3824.5669,  3250.8965, -1562.5955,  4669.1475, -2415.9492],
       grad_fn=<SelectBackward0>)


In [2]:
import jax
import jax.numpy as jnp

In [ ]:

@jax.jit
def f(x,y):
    print(type(x))
    print(type(y))
    return x+y

f(1,2)

<class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>


Array(1, dtype=int32, weak_type=True)